In [36]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, minmax_scale, scale
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [37]:
seed = 0

In [38]:
DATASETX = "./data/prep/HotelReservationsPreparedCleanX.csv"
DATASETY = "./data/prep/HotelReservationsY.csv"

In [39]:
df_x = pd.read_csv(DATASETX)
df_y = pd.read_csv(DATASETY)

In [40]:
df_x.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
0,0.0,-0.6,-1.000000,-0.882353,-1.0,1.0,-1.0,-0.990971,0.454545,-0.933333,0.0,-1.0,-1.0,-1.0,-0.044444,-0.6
1,0.0,-1.0,-1.000000,-0.764706,0.0,-1.0,0.0,-0.823928,0.272727,-0.133333,-1.0,-1.0,-1.0,-1.0,-0.624074,-1.0
2,0.0,-1.0,-0.428571,-0.764706,-1.0,-1.0,0.0,-0.936795,1.000000,0.733333,0.0,-1.0,-1.0,-1.0,-0.603704,-1.0
3,0.0,-1.0,-1.000000,-0.882353,0.0,-1.0,0.0,-0.548533,0.636364,0.000000,-1.0,-1.0,-1.0,-1.0,-0.600000,-1.0
4,-0.5,-1.0,-1.000000,-0.882353,0.0,-1.0,0.0,-0.002257,0.272727,-0.266667,-1.0,-1.0,-1.0,-1.0,-0.729630,-1.0


In [41]:
df_y.head()

,booking_status
0,1
1,1
2,1
3,0
4,1


In [42]:
df = pd.concat([df_x, df_y], axis=1)
df.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,0.0,-0.6,-1.000000,-0.882353,-1.0,1.0,-1.0,-0.990971,0.454545,-0.933333,0.0,-1.0,-1.0,-1.0,-0.044444,-0.6,1
1,0.0,-1.0,-1.000000,-0.764706,0.0,-1.0,0.0,-0.823928,0.272727,-0.133333,-1.0,-1.0,-1.0,-1.0,-0.624074,-1.0,1
2,0.0,-1.0,-0.428571,-0.764706,-1.0,-1.0,0.0,-0.936795,1.000000,0.733333,0.0,-1.0,-1.0,-1.0,-0.603704,-1.0,1
3,0.0,-1.0,-1.000000,-0.882353,0.0,-1.0,0.0,-0.548533,0.636364,0.000000,-1.0,-1.0,-1.0,-1.0,-0.600000,-1.0,0
4,-0.5,-1.0,-1.000000,-0.882353,0.0,-1.0,0.0,-0.002257,0.272727,-0.266667,-1.0,-1.0,-1.0,-1.0,-0.729630,-1.0,1


In [43]:
df = shuffle(df, random_state=seed)
df.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
17405,0.0,-1.0,-0.714286,-0.882353,-1.0,1.0,0.0,-0.959368,0.272727,-0.200000,0.0,-1.0,-1.0,-1.0,-0.388889,0.2,1
25403,0.0,-1.0,-1.000000,-0.882353,-1.0,-1.0,0.0,-0.936795,0.636364,0.400000,0.0,-1.0,-1.0,-1.0,-0.407407,-1.0,0
13544,0.5,-1.0,-1.000000,-0.647059,-1.0,-1.0,1.0,-0.638826,0.272727,-0.866667,0.0,-1.0,-1.0,-1.0,-0.499000,-0.6,1
31848,0.0,-1.0,-0.428571,-0.764706,-1.0,-1.0,1.0,-0.864560,0.636364,0.000000,0.0,-1.0,-1.0,-1.0,-0.560815,-0.6,1
9691,-0.5,-1.0,-0.428571,-0.411765,-1.0,-1.0,1.0,-0.977427,0.636364,-0.333333,1.0,-1.0,-1.0,-1.0,-0.592593,-1.0,1


In [44]:
# Dividimos el dataframe en train y test en una proporción de 80/20
X_train, X_test, y_train, y_test = train_test_split(df, df["booking_status"], test_size=0.2, random_state=seed)

# Dividimos el conjunto de prueba en test y validation en una proporción de 50/50
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [45]:
print("Train: ", X_train.shape)
print("Test: ", X_test.shape)
print("Validation: ", X_val.shape)

Train:  (29016, 17)
Test:  (3627, 17)
Validation:  (3627, 17)
